# Local binary pattern - on artificial data
Came from 2020-09-04_Local_binary_pattern_with_covid
and came from 2020-09-29_Local_binary_pattern_with_covid
and came from 2020-10-11_Local_binary_pattern_with_covid

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
import tifffile
import os
import pandas as pd

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if False:
    fileroot = '/home/ubuntu/Documents/'

colors_new=['#0072b2','#009e73','#d55e00', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf 
import time

Loading Linux


In [2]:
import matplotlib
import matplotlib.patheffects as path_effects
from functools import partial
from skimage.feature import local_binary_pattern
from importlib import reload
from itertools import repeat
from datetime import datetime
from tqdm.notebook import trange

In [3]:
#reload(uf)

In [4]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [5]:
#This should be the folder where the LBP values for the images are output
#Should be chosen to be a folder that already exists (but can be empty)
directory_out_top = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/'

In [6]:
directory_in_imgs = fileroot + 'VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06d_image_conversion_and_crop_to_small_version/'
#imgs = os.listdir(directory_in_imgs)
#imgs.remove('Thumbs.db')
#imgs.sort(key=partial(uf.getvaluefromstringbest, variable='MS', ending='.', mydtype=int))
imgs = ['V11J11-099__Z4_FO3_01__A1_small.tif']
#imgs = [img for img in imgs if '.npy' in img]
imgs

['V11J11-099__Z4_FO3_01__A1_small.tif']

In [7]:
#this is one of the two parameters of the process
#I normally set the highest radius in radius_list to be the highest number that is smaller than the patchsize I choose
radius_list = [round(1.499*1.327**(float(x))) for x in range(0, 15)]
print(radius_list); len(radius_list)

[1, 2, 3, 4, 5, 6, 8, 11, 14, 19, 25, 34, 45, 59, 79]


15

In [8]:
#directory_metadata = fileroot + '0ExternalData/2022-05-10_Newcastle_University_iridium_covid_UK_CIC_dataset/'
#filename_metadata = 'Cross_site_Technology_ROI_Summary_with_filenames.xlsx'
#df2 = pd.read_excel(directory_metadata + filename_metadata, engine='openpyxl')
#df2

In [9]:
#df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all

In [10]:
#result = df_all.merge(df2, how='outer', indicator=True)
#result[result['_merge'] != 'both']

In [11]:
#This part is creating all the different conditions for LBP to run over
#This is mostly used to run over 

def get_dims_tiff(filepath):
    myimg = skimage.io.imread(filepath)
    return myimg.shape

df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all['dims'] = list(map(uf.get_dims_from_numpy_file, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
df_all['dims'] = list(map(get_dims_tiff, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
#df_all['dims_superpixel'] = list(map(uf.get_cropped_superpixel_dims, df_all['dims'], repeat(1)))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(radius_list, columns=['radius']))

#df_all['n_points'] = [int(32/5*r) for r in df_all['radius']]
df_all['n_points'] = [math.ceil(2*math.pi*r) for r in df_all['radius']]
df_all['method'] = ['uniform']*len(df_all)
df_channel = pd.DataFrame([0, 1, 2], columns=['channel'])

df_all = uf.create_df_cross_for_separate_dfs(df_all, df_channel)

df_all.sort_values(['n_points', 'Filenames'], inplace=True)
df_all.reset_index(drop=True, inplace=True)

df_all

,Filenames,dims,radius,n_points,method,channel
0,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,7,uniform,0
1,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,7,uniform,1
2,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",1,7,uniform,2
3,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,13,uniform,0
4,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,13,uniform,1
5,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",2,13,uniform,2
6,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,19,uniform,0
7,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,19,uniform,1
8,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",3,19,uniform,2
9,V11J11-099__Z4_FO3_01__A1_small.tif,"(5539, 3739, 3)",4,26,uniform,0


In [12]:
def get_numpy_datatype_unsigned_int(largest_value):
    if largest_value <= 255:
        print('In lower 255')
        value = np.uint8
    elif largest_value <= 65535:
        value = np.uint16
    else:
        value = 0
    return value

mymax = np.max(df_all['n_points']); print(mymax)
mydtype = get_numpy_datatype_unsigned_int(mymax); print(mydtype)

497
<class 'numpy.uint16'>


In [13]:
for i in trange(len(df_all)):
#    print(i, '/', len(df_all))
    filename = df_all['Filenames'][i]
    filename_out = filename.replace('.tif', '.npy')
#    out_dims = df_all['dims_superpixel'][i]
    radius = df_all['radius'][i]
    n_points = df_all['n_points'][i]
    method = df_all['method'][i]
    channel = df_all['channel'][i]

    start = datetime.now()
#    print(start)
    
    sub_directory_out = 'Channel_' + str(channel) + '_METHOD_'+ method + '_npoints_' + str(n_points) + '_radius_' + str(radius) + '/'
#    print(directory_out_top+sub_directory_out)
    if not os.path.exists(directory_out_top+sub_directory_out):
        os.mkdir(directory_out_top+sub_directory_out)
        
    if os.path.exists(directory_out_top+sub_directory_out+filename_out):
        print('Already exists: ', directory_out_top+sub_directory_out+filename_out)
        continue
    
    img = skimage.io.imread(directory_in_imgs + filename)[:,:,channel]
    lbp = local_binary_pattern(img, n_points, radius, method).astype(mydtype)
    np.save(directory_out_top+sub_directory_out+filename_out, lbp)
#    print(datetime.now() - start)

  0%|          | 0/45 [00:00<?, ?it/s]

Already exists:  /home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/Channel_0_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy
Already exists:  /home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/Channel_1_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy
Already exists:  /home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/Channel_2_METHOD_uniform_npoints_7_radius_1/V11J11-099__Z4_FO3_01__A1_small.npy
Already exists:  /home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/Channel_0_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy
Already exists:  /home/jovyan/VU_TEX/playground_Ben/2023-07-05_Thymus/2023-07-06f_LBP_values_for_image_small/Channel_1_METHOD_uniform_npoints_13_radius_2/V11J11-099__Z4_FO3_01__A1_small.npy
Already exists:  /home/jovyan/VU_TEX/playground_Ben/2

## 